### Importing Class Similarities Data

First we will import some modules that we might need 😉

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.metrics import mean_squared_error



##Part 1: Exploring the Data


Now lets read in the data from the ClassSimilarities_2025Spring.csv matrix. [Click here to download the data](https://drive.google.com/uc?download&id=1KHm51Dv09t9bcSaJbSn8eCBLY8ZUsdj2)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# read in data
sims = pd.read_csv('ClassSimilarities_2025Spring.csv')
sims.head()


In [ ]:
# frequency table of values for Section
sims['Section'].value_counts()


Uncomment the line for your class and create the `class_df` data frame
Call your matrix "class_df"


In [ ]:

#class_df = sims[sims['Section'] == 'Section 1 (MW 930am)']
#class_df = sims[sims['Section'] == 'Section 2 (MW 11am)']
#class_df = sims[sims['Section'] == 'Section 30 (W 6pm)']

class_df.shape

Explore the features of interest.  
Sort by the mean to see which features are most (and least) popular.  Maybe even sort hightest to lowest!


In [ ]:
# Place code here


Plot barplots to visualize the distributions of each of the features.

Suggestion - define the columns of interest as `feature_cols` and use a for loop to iterate over all of the features of interest and plot a barplot.

Try and get the barplot to plot in order from 1 to 5 on the x-axis. (can use "reindex" for this)

In [ ]:
# Place code here



Plot a heatmap of the correlations to see which features are most positively and negatively correlated.  Use the `.corr()` function in seaborn and `sns.heatmap`.  Any interesting results here?

*you can play with colorbrewer colors here!  My favorite is cmap='RdBu_r' and dont forget to set the limits to (-1,1) using `vmax` and `vmin`*

In [ ]:
# Place code here

Now plot a histogram of caffeine cups per week (`CoffeeTeaSoda`).  How many zeros are there?

How many people dont drink any caffeine? 😲

In [ ]:
# Place code here



## Part 2: Looking at distances between students

One helpful suggestions would be to set *NYUID* as the index so that you can easily extract the data for any given  NYUID.

In [ ]:

# Set 'NYUID' as the index - this allows you to call the row by the NYUID
class_df.set_index('NYUID', inplace=True)


Use `sklearn.pairwise_distances` (already installed above) to calculate a distance matrix "dist_matrix"  (It may be helpful to convert the dist_matrix into a data frame using pd.DataFrame)

Use Manhattan distance because it will be more iterpretable.

Identify your data with your NYUID and see if you can find the three most similar students to you and the three least similar students to you

In [ ]:
# Place code here

input_NYUID='YOUR_ID_HERE'

Find the three most similar students to you and the three least similar students to you.

Remember that the most similar student will be the student themself!  (distance = 0) so you will have to account for that.

also you might want to check to make sure that the input_NYUID is actuall in the index or else you will get an error.

In [ ]:
if (input_NYUID in class_df.index):

    # Get the distances for the input NYUID

    # Get the 3 most similar students (excluding the input student itself)

    # Get the 3 least similar students

    # Print the NYUID and Name of the most and least similar students

else:
    print('NYUID not found in the dataset')


Which two people in the class are overall the closest?  Because we use Manhattan distance, this distance is interpreted as the total sum of the absolute differences of the 24 features.

You can look for the smallest value in the matrix, but you need to account for the fact that the diagonals are zero...

this one is tricky, you may need to use `unravel_index` to find the row and col in the matrix with the lowest value...(see solution - although there are other ways to do this as well)

In [ ]:
# Place code here


Which two people in the class are the furthest?

In [ ]:
# Place code here

##Part 3: K nearest neighbors for predicting targets##

We want to know if the preferences data has value in predicting the Coffee consumption or the Hours Sleep.

Since we dont have a lot of data in each class, use the entire `sims` data (both classes) and split into 80/20 and fit a knn with k=5 (aka `KNeighborsRegressor(n_neighbors=5)`


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

# Select the features and target
X = sims[feature_cols]
y = sims['CoffeeTeaSoda']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(???)

# Create a KNeighborsRegressor with k=5

# Place Code Here

Make a scatterplot of predicted values for the test set on the x-axis and actual values on the y-axis.

Does the prediction seem any good?

In [ ]:
# Place code here

Using a for-loop - find the best k via RMSE.

In [ ]:
#yikes this is not good!  But lets calcuate the best k anyway
from sklearn.metrics import mean_squared_error

for k in range(1, 21):
  #Place code here


##Part 4: Clustering


Lets perform hierarchical clustering to find the "10-cluster solution" and print out the clusters.

Did any of you end up in clusters with your project team members?

All code provided...




In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Create an AgglomerativeClustering model with 6 clusters
cluster = AgglomerativeClustering(n_clusters=10, linkage='ward')

# Fit the model to the data and predict the cluster labels
cluster_labels = cluster.fit_predict(class_df[feature_cols])
student_clusters = class_df.assign(Cluster=cluster_labels)



In [ ]:

for cluster, data in student_clusters.groupby('Cluster'):
    print(f"Cluster {cluster}:")
    print(data.Name.tolist())
    print()

Plot the dendrogram.

Play around with the following  parameters:
in the linkage function : `method` and `metric`

in the dendrogram: `orientation` - for the direction of the plot
                : `color_threshold` - for how the plot is colored showing different clusters

can use `labels=class_df.index` to show NYUID instead of names...


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

# Create a linkage matrix
linked = linkage(class_df[feature_cols], 'ward')

# Plot the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linked, orientation='top', labels=class_df.Name, distance_sort='descending', show_leaf_counts=True)
plt.show()

For fun - we can also cluster the *features* to see which are most similar...by passing the feature correlation matrix into the `linked` function

In [ ]:
# Calculate the correlation matrix
corr = class_df[feature_cols].corr()

# Create a linkage matrix based on the correlation matrix
linked = linkage(corr, 'ward')

# Create a dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linked, labels=corr.columns, orientation='right',color_threshold = 2)
plt.show()